In [1]:
# wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [2]:
from factue.methods.llm_calls import load_template_list
for x in load_template_list(job='persuasion', step="detect", prompt_version='v001').keys():
    print(f'"{x}"')

"Name_Calling-Labeling"
"Guilt_by_Association"
"Doubt"
"Appeal_to_Hypocrisy"
"Questioning_the_Reputation"
"Flag_Waving"
"Appeal_to_Authority"
"Appeal_to_Popularity"
"Appeal_to_Values"
"Appeal_to_Fear-Prejudice"
"Straw_Man"
"Red_Herring"
"Whataboutism"
"Appeal_to_Pity"
"Causal_Oversimplification"
"False_Dilemma-No_Choice"
"Consequential_Oversimplification"
"False_Equivalence"
"Slogans"
"Conversation_Killer"
"Appeal_to_Time"
"Loaded_Language"
"Obfuscation-Vagueness-Confusion"
"Exaggeration-Minimisation"
"Repetition"


In [3]:
import pandas as pd
from pathlib import Path
import os
root = Path("data/llm_output/persuasion")
# Example path template
files = root.rglob("*/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]

In [4]:
df.head(3)

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,description,verdict,is_valid,extra_properties,illegal_value,pred,gold,source_file,error,split
0,pl_eu_12_06_2024_n01.txt,0,135,PL,Wicemarszałek Włodzimierz Czarzasty:\nBardzo p...,False,[],train,[],[],...,[The example does not contain Appeal to Popula...,"[0, 0, 0, 0, 0]","[True, True, True, True, True]","[None, None, None, None, None]","[None, None, None, None, None]",0,0,data/llm_output/persuasion/detect/LLAMA_31_8B/...,NaN,train
1,pl_defence_22_05_2024_n01.txt,3608,3857,PL,Pan wspominał o ABCS – Integrated Battle Comma...,False,[],train,[],[],...,[The example contains Appeal to Popularity bec...,"[1, 0, 0, 1, 1]","[True, True, True, True, True]","[None, None, None, None, None]","[None, None, None, None, None]",1,0,data/llm_output/persuasion/detect/LLAMA_31_8B/...,NaN,train
2,pl_current_affairs_09_01_2025_n03.txt,1808,1984,PL,(Poseł Anna Gembicka: Decyzja została zmienion...,True,[Repetition],train,[],[],...,[The text does not explicitly contain Appeal t...,"[0, 0, 0, 0, 0]","[True, True, True, True, True]","[None, None, None, None, None]","[None, None, None, None, None]",0,0,data/llm_output/persuasion/detect/LLAMA_31_8B/...,NaN,train


In [5]:
df.pred.value_counts()

pred
 0    9728
 1    8382
-1     143
Name: count, dtype: int64

In [6]:
df[df.split=='dev'].groupby(["prompt_name","text_lang","model_name","split"])['filename'].agg("count").reset_index()

,prompt_name,text_lang,model_name,split,filename
0,Appeal_to_Authority,BG,gpt-4o-mini,dev,184
1,Appeal_to_Authority,BG,llama3.1:8b,dev,164
2,Appeal_to_Authority,PL,gpt-4o-mini,dev,114
3,Appeal_to_Authority,PL,llama3.1:8b,dev,114
4,Appeal_to_Authority,RU,gpt-4o-mini,dev,90
...,...,...,...,...,...
107,Straw_Man,SI,llama3.1:8b,dev,41
108,Whataboutism,BG,llama3.1:8b,dev,164
109,Whataboutism,PL,llama3.1:8b,dev,114
110,Whataboutism,RU,llama3.1:8b,dev,90


In [7]:
def normalize_binary(x):
    return 1 if str(x).strip().lower() in {'1', 'true'} else 0

In [8]:
df['source_file'].value_counts().sort_index()
df['gold'] = df['gold'].apply(normalize_binary)
df['pred'] = df['pred'].apply(normalize_binary)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]

In [9]:
df.prompt_name.value_counts()

prompt_name
Appeal_to_Popularity          1487
Doubt                         1487
Appeal_to_Hypocrisy           1477
Appeal_to_Authority           1477
Flag_Waving                   1467
Name_Calling-Labeling         1467
Appeal_to_Fear-Prejudice      1427
Appeal_to_Values              1407
Straw_Man                     1266
Guilt_by_Association          1149
Whataboutism                  1058
Red_Herring                   1018
Questioning_the_Reputation     668
Exaggeration-Minimisation      429
Repetition                     429
Loaded_Language                419
Appeal_to_Pity                 121
Name: count, dtype: int64

In [10]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# Define a function to compute metrics for a group
# Assume df has columns: prompt_name, text_lang, gold, pred
rows = []

def is_valid_label(x):
    return x in (0, 1)

rows = []

for prompt_name in df['prompt_name'].unique():
    for text_lang in df.loc[df['prompt_name'] == prompt_name, 'text_lang'].unique():
        for model_name in df.loc[(df['prompt_name'] == prompt_name) & (df['text_lang'] == text_lang), 'model_name'].unique():
            group = df[
                (df['prompt_name'] == prompt_name) &
                (df['text_lang'] == text_lang) &
                (df['model_name'] == model_name)
            ]
            if len(group) > 0:
                row = {
                    'prompt_name': prompt_name,
                    'text_lang': text_lang,
                    'model_name': model_name,
                    'accuracy': accuracy_score(group['gold'], group['pred']),
                    'precision': precision_score(group['gold'], group['pred'], zero_division=0),
                    'recall': recall_score(group['gold'], group['pred'], zero_division=0),
                    'f1': f1_score(group['gold'], group['pred'], zero_division=0),
                    'support': len(group)
                }
                rows.append(row)

results = pd.DataFrame(rows)
results[['accuracy', 'precision','recall', 'f1']] = results[['accuracy', 'precision','recall', 'f1']].round(2)
results

,prompt_name,text_lang,model_name,accuracy,precision,recall,f1,support
0,Appeal_to_Popularity,PL,llama3.1:8b,0.63,0.15,0.66,0.24,316
1,Appeal_to_Popularity,PL,gpt-4o-mini,0.84,0.36,0.27,0.31,114
2,Appeal_to_Popularity,RU,llama3.1:8b,0.42,0.02,1.00,0.04,257
3,Appeal_to_Popularity,RU,gpt-4o-mini,0.94,0.17,1.00,0.29,90
4,Appeal_to_Popularity,SI,llama3.1:8b,0.62,0.30,0.87,0.44,87
...,...,...,...,...,...,...,...,...
107,Repetition,BG,gpt-4o-mini,0.64,0.07,0.83,0.13,184
108,Loaded_Language,PL,gpt-4o-mini,0.55,0.34,0.93,0.50,114
109,Loaded_Language,RU,gpt-4o-mini,0.35,0.06,0.60,0.10,80
110,Loaded_Language,SI,gpt-4o-mini,0.76,0.55,0.92,0.69,41


In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

# train

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt

# Melt the results dataframe
metrics_df = results[results['split']=='train'].melt(
    id_vars=['prompt_name', 'text_lang', 'model_name'],
    value_vars=['accuracy', 'precision', 'recall', 'f1'],
    var_name='metric',
    value_name='score'
)

# Create FacetGrid
g = sns.FacetGrid(
    metrics_df, 
    row="text_lang", 
    col="metric", 
    sharex=True, 
    sharey=False, 
    height=4, 
    aspect=1.5
)

# Use map_dataframe and assign `hue` in the barplot call
g.map_dataframe(
    sns.barplot, 
    x="score", 
    y="prompt_name", 
    hue="model_name", 
    errorbar=None, 
    palette='muted',
    legend=False  # Disable auto-legend here
)

# Add a single legend to the full figure
g.add_legend(title='Model')
g.set_axis_labels("Score", "Technique ID")
g.set_titles(row_template="{row_name}", col_template="{col_name}")
for ax in g.axes.flatten():
    ax.grid(axis='x', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


KeyError: 'split'

# total

In [13]:
# Metrics
acc = accuracy_score(df['gold'], df['pred'])
precision = precision_score(df['gold'], df['pred'], zero_division=0)
recall = recall_score(df['gold'], df['pred'], average='binary')
f1 = f1_score(df['gold'], df['pred'], average='binary')

print(f"Accuracy: {acc:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")

Accuracy: 0.588
Precision: 0.125
Recall: 0.848
F1-score: 0.218
